# ML Final Project


In [64]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [65]:
## Imports
import numpy as np
# from sklearn.linear_model import train_test_split

In [66]:
## Load data
path = '/content/drive/MyDrive/ML Final Project/data/'
data_amzn = np.loadtxt(path + 'amazon_cells_labelled.txt', dtype='object',delimiter='\t')
x_amzn = [data[0] for data in data_amzn]
y_amzn = [int(data[1]) for data in data_amzn]

data_imdb = np.loadtxt(path + 'imdb_labelled.txt', dtype='object',delimiter='\t')
x_imdb = [data[0] for data in data_imdb]
y_imdb = [int(data[1]) for data in data_imdb]

data_yelp = np.loadtxt(path + 'yelp_labelled.txt', dtype='object',delimiter='\t')
x_yelp = [data[0] for data in data_yelp]
y_yelp = [int(data[1]) for data in data_yelp]

train_cutoff = 800
dev_cutoff = 900

x_train = x_amzn[0:train_cutoff] + x_imdb[0:train_cutoff] + x_yelp[0:train_cutoff]
y_train = y_amzn[0:train_cutoff] + y_imdb[0:train_cutoff] + y_yelp[0:train_cutoff]

x_dev = x_amzn[train_cutoff:dev_cutoff] + x_imdb[train_cutoff:dev_cutoff] + x_yelp[train_cutoff:dev_cutoff]
y_dev = y_amzn[train_cutoff:dev_cutoff] + y_imdb[train_cutoff:dev_cutoff] + y_yelp[train_cutoff:dev_cutoff]

x_test = x_amzn[dev_cutoff:] + x_imdb[dev_cutoff:] + x_yelp[dev_cutoff:]
y_test = y_amzn[dev_cutoff:] + y_imdb[dev_cutoff:] + y_yelp[dev_cutoff:]

In [67]:
## Preprocess data
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(x_train)
X_dev = vectorizer.transform(x_dev)
X_test = vectorizer.transform(x_test)


In [68]:
import tensorflow as tf
from keras.preprocessing import sequence

vocab_size = 10000
encoded_training = [tf.keras.preprocessing.text.one_hot(d, vocab_size) for d in x_train] 
encoded_dev = [tf.keras.preprocessing.text.one_hot(d, vocab_size) for d in x_dev]
encoded_test = [tf.keras.preprocessing.text.one_hot(d, vocab_size) for d in x_test]
max_words = 500

X_train = sequence.pad_sequences(encoded_training, maxlen=max_words)
X_dev = sequence.pad_sequences(encoded_dev, maxlen=max_words)
X_test = sequence.pad_sequences(encoded_test, maxlen= max_words)


In [70]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

from tensorflow import keras
from tensorflow.keras import layers

embedding_size=64
model=Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

In [71]:
opt = keras.optimizers.Adam(learning_rate=0.011)

model.compile(loss='binary_crossentropy', 
             optimizer=opt, 
             metrics=['accuracy'])

In [72]:
batch_size = 60
num_epochs = 3

y_dev = np.array(y_dev)
y_train = np.array(y_train)



model.fit(X_train, y_train, validation_data=(X_dev, y_dev), batch_size=batch_size, epochs=num_epochs)

Epoch 1/3
40/40 [==============================] - 22s 549ms/step - loss: 0.6501 - accuracy: 0.6400 - val_loss: 0.5571 - val_accuracy: 0.6967
Epoch 2/3
40/40 [==============================] - 22s 542ms/step - loss: 0.3091 - accuracy: 0.8733 - val_loss: 0.5681 - val_accuracy: 0.7500
Epoch 3/3
40/40 [==============================] - 21s 537ms/step - loss: 0.1176 - accuracy: 0.9654 - val_loss: 0.7064 - val_accuracy: 0.7567


In [73]:
model.evaluate(X_test, np.array(y_test))

10/10 [==============================] - 1s 79ms/step - loss: 0.4875 - accuracy: 0.8227


[0.48747390508651733, 0.8227424621582031]

In [74]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
# print(len(y_test))

predictions = model.predict(X_test)  
# print(predictions)

predictions = np.where(predictions > .5,1,0)

# print(confusion_matrix(np.array(y_test), predictions))
print(classification_report(np.array(y_test), predictions))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       186
           1       0.79      0.73      0.76       113

    accuracy                           0.82       299
   macro avg       0.81      0.80      0.81       299
weighted avg       0.82      0.82      0.82       299

